# Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import time
import copy
# import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg
import random

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms
import torch.optim as optim

from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torchvision.models.feature_extraction import create_feature_extractor
import torchvision.models as models
from torchsummary import summary
from torch.utils.data import RandomSampler
from torchvision.utils import make_grid

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Order model

**Downloading the model**

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('f device :{}', device)
model_ft = models.efficientnet_b0(weights='EfficientNet_B0_Weights.IMAGENET1K_V1') #(pretrained=True)

#excluding the fully connected layers at the end
model_ft.classifier = nn.Sequential(*list(model_ft.classifier.children())[:-3])
#torch.save(model_ft.state_dict(), 'efficientnet_pretrained.ckpt')

f device :{} cpu


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth


  0%|          | 0.00/20.5M [00:00<?, ?B/s]

**Add our layer with 450 outputs**

In [ ]:
model_ft.classifier = nn.Sequential(
    nn.Linear(in_features=1280 ,out_features=450),
    nn.Softmax()
)
summary (model_ft, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              SiLU-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
              SiLU-6         [-1, 32, 112, 112]               0
 AdaptiveAvgPool2d-7             [-1, 32, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]             264
              SiLU-9              [-1, 8, 1, 1]               0
           Conv2d-10             [-1, 32, 1, 1]             288
          Sigmoid-11             [-1, 32, 1, 1]               0
SqueezeExcitation-12         [-1, 32, 112, 112]               0
           Conv2d-13         [-1, 16, 112, 112]             512
      BatchNorm2d-14         [-1, 16, 1

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


# Freeze training for all layers

**Functions for freezing and unfreezing relavant parts of the architecture**

In [ ]:
def freeze_backbone(model):
    for param in model.features.parameters():
        param.requires_grad = False

def freeze_head(model):
    for param in model.classifier.parameters():
        param.requires_grad = False

def unfreeze_backbone(model):
    for param in model.features.parameters():
        param.requires_grad = True

def unfreeze_head(model):
    for param in model.classifier.parameters():
        param

**Freeze parameters**

In [ ]:
freeze_backbone(model_ft)
unfreeze_head(model_ft)
summary (model_ft, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              SiLU-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
              SiLU-6         [-1, 32, 112, 112]               0
 AdaptiveAvgPool2d-7             [-1, 32, 1, 1]               0
            Conv2d-8              [-1, 8, 1, 1]             264
              SiLU-9              [-1, 8, 1, 1]               0
           Conv2d-10             [-1, 32, 1, 1]             288
          Sigmoid-11             [-1, 32, 1, 1]               0
SqueezeExcitation-12         [-1, 32, 112, 112]               0
           Conv2d-13         [-1, 16, 112, 112]             512
      BatchNorm2d-14         [-1, 16, 1

# Create Class

In [ ]:
class BirdDataset(Dataset):
    
    def __init__(self, imgs_list, class_to_int, transforms = None):
        super().__init__()
        self.imgs_list = imgs_list
        self.class_to_int = class_to_int
        self.transforms = transforms
        
        
    def __getitem__(self, index):
        image_path = self.imgs_list[index]
        
        #Reading image
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image = np.around(image /255.0,3)
        
        #Retriving class label
        label = image_path.split("/")[-2]
        label = self.class_to_int[label]
        
        #Applying transforms on image
        if self.transforms:
            image = self.transforms(image)
        
        return image, label


    def __len__(self):
        return len(self.imgs_list)

# Set hyper-parameters

In [ ]:
train_dir = "/content/drive/MyDrive/Birds_implemention/train/"
test_dir = "/content/drive/MyDrive/Birds_implemention/test/"
val_dir = "/content/drive/MyDrive/Birds_implemention/valid/"
learning_rate = 1e-3
epochs = 2 # to_check
num_of_classes = 450
batch_Size = 10
criterion = nn.CrossEntropyLoss()
optimazer = optim.Adam(model_ft.parameters(), lr=learning_rate)
TRAIN = 'train'
VAL = 'valid'
TEST = 'test'

In [ ]:
classes = os.listdir(train_dir)
#print(classes)

['ABYSSINIAN GROUND HORNBILL', 'ABBOTTS BOOBY', 'ABBOTTS BABBLER', 'AFRICAN CROWNED CRANE', 'AFRICAN OYSTER CATCHER', 'AFRICAN PIED HORNBILL', 'AFRICAN FIREFINCH', 'ALTAMIRA YELLOWTHROAT', 'ALEXANDRINE PARAKEET', 'ALPINE CHOUGH', 'AFRICAN EMERALD CUCKOO', 'ALBERTS TOWHEE', 'ALBATROSS', 'AMERICAN AVOCET', 'AMETHYST WOODSTAR', 'AMERICAN KESTREL', 'AMERICAN COOT', 'AMERICAN GOLDFINCH', 'AMERICAN REDSTART', 'AMERICAN FLAMINGO', 'AMERICAN PIPIT', 'AMERICAN WIGEON', 'AMERICAN BITTERN', 'ANDEAN GOOSE', 'APAPANE', 'ANTBIRD', 'ANDEAN LAPWING', 'ANTILLEAN EUPHONIA', 'ANIANIAU', 'ANDEAN SISKIN', 'APOSTLEBIRD', 'ANNAS HUMMINGBIRD', 'ANHINGA', 'ARARIPE MANAKIN', 'ASHY STORM PETREL', 'ASIAN CRESTED IBIS', 'AUCKLAND SHAQ', 'AUSTRAL CANASTERO', 'AZARAS SPINETAIL', 'ASIAN DOLLARD BIRD', 'AZURE BREASTED PITTA', 'AVADAVAT', 'ASHY THRUSHBIRD', 'AUSTRALASIAN FIGBIRD', 'AZURE JAY', 'AZURE TANAGER', 'AZURE TIT', 'BALTIMORE ORIOLE', 'BANDED BROADBILL', 'BAND TAILED GUAN', 'BANANAQUIT', 'BALI STARLING', 'BALD 

In [ ]:
# Number of classes
class_to_int = {classes[i] : i for i in range(len(classes))}
#print(class_to_int)

{'ABYSSINIAN GROUND HORNBILL': 0, 'ABBOTTS BOOBY': 1, 'ABBOTTS BABBLER': 2, 'AFRICAN CROWNED CRANE': 3, 'AFRICAN OYSTER CATCHER': 4, 'AFRICAN PIED HORNBILL': 5, 'AFRICAN FIREFINCH': 6, 'ALTAMIRA YELLOWTHROAT': 7, 'ALEXANDRINE PARAKEET': 8, 'ALPINE CHOUGH': 9, 'AFRICAN EMERALD CUCKOO': 10, 'ALBERTS TOWHEE': 11, 'ALBATROSS': 12, 'AMERICAN AVOCET': 13, 'AMETHYST WOODSTAR': 14, 'AMERICAN KESTREL': 15, 'AMERICAN COOT': 16, 'AMERICAN GOLDFINCH': 17, 'AMERICAN REDSTART': 18, 'AMERICAN FLAMINGO': 19, 'AMERICAN PIPIT': 20, 'AMERICAN WIGEON': 21, 'AMERICAN BITTERN': 22, 'ANDEAN GOOSE': 23, 'APAPANE': 24, 'ANTBIRD': 25, 'ANDEAN LAPWING': 26, 'ANTILLEAN EUPHONIA': 27, 'ANIANIAU': 28, 'ANDEAN SISKIN': 29, 'APOSTLEBIRD': 30, 'ANNAS HUMMINGBIRD': 31, 'ANHINGA': 32, 'ARARIPE MANAKIN': 33, 'ASHY STORM PETREL': 34, 'ASIAN CRESTED IBIS': 35, 'AUCKLAND SHAQ': 36, 'AUSTRAL CANASTERO': 37, 'AZARAS SPINETAIL': 38, 'ASIAN DOLLARD BIRD': 39, 'AZURE BREASTED PITTA': 40, 'AVADAVAT': 41, 'ASHY THRUSHBIRD': 42, 'A

**Append images to dataFrame**

In [ ]:
train_imgs = []
valid_imgs = []
test_imgs = []

for c in classes:
    
    for img in os.listdir(train_dir + c):
        train_imgs.append(train_dir + c + "/" + img)
    
    for img in os.listdir(val_dir + c):
        valid_imgs.append(val_dir + c + "/" + img)
        
    for img in os.listdir(test_dir + c):
        test_imgs.append(test_dir + c + "/" + img)

train_imgs = pd.DataFrame(train_imgs)
valid_imgs = pd.DataFrame(valid_imgs)
test_imgs = pd.DataFrame(test_imgs)

In [ ]:
for c,i in enumerate(train_imgs[c]):
  print(c,i)
print(train_imgs)

KeyError: ignored

**Save as csv file**

In [ ]:
train_imgs.to_csv("/content/drive/MyDrive/Birds_implemention/train_imgs.csv")
valid_imgs.to_csv("/content/drive/MyDrive/Birds_implemention/valid_imgs.csv")
test_imgs.to_csv("/content/drive/MyDrive/Birds_implemention/test_imgs.csv")

# Construct dataset and dataloader

**Dataset**

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Resize((224,224))])
# Method 2: Using Dataset Class
train_dataset = BirdDataset(train_imgs, class_to_int, transform)
valid_dataset = BirdDataset(valid_imgs, class_to_int, transform)
test_dataset = BirdDataset(test_imgs, class_to_int, transform)

#Data Loader - using Sampler (YT Video)
train_random_sampler = RandomSampler(train_dataset)
valid_random_sampler = RandomSampler(valid_dataset)
test_random_sampler = RandomSampler(test_dataset)

**Dataloader**

In [ ]:
def data_loader(data, batch, random_sampler):
  return DataLoader(
    dataset = data,
    batch_size = batch,
    sampler = random_sampler,
    num_workers = 0,
  )

In [ ]:
#Shuffle Argument is mutually exclusive with Sampler!

train_data_loader = data_loader(train_dataset, 10, train_random_sampler)
valid_data_loader = data_loader(valid_dataset, 10, valid_random_sampler)
test_data_loader = data_loader(test_dataset, 10, test_random_sampler)

# for x in [TRAIN, VAL, TEST]:
#     print("Loaded {} images under {}".format(dataset_sizes[x], x))

In [ ]:
#Shuffle Argument is mutually exclusive with Sampler!
train_data_loader = DataLoader(
    dataset = train_dataset,
    batch_size = 10,
    sampler = train_random_sampler,
    num_workers = 0,
)

valid_data_loader = DataLoader(
    dataset = valid_dataset,
    batch_size = 10,
    sampler = valid_random_sampler,
    num_workers = 0,
)

test_data_loader = DataLoader(
    dataset = test_dataset,
    batch_size = 10,
    sampler = test_random_sampler,
    num_workers = 0,
)

# To device

In [ ]:
model_ft.to(device)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

# Train Model

In [ ]:
def train_model(model_tr, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    # save first the best model until now
    # best_model_wts = copy.deepcopy(model_tr.state_dict())
    best_acc = 0.0
    
    avg_loss = 0
    avg_acc = 0
    avg_loss_val = 0
    avg_acc_val = 0
    
    train_batches = len(train_data_loader)
    val_batches = len(valid_data_loader)
    
    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch, num_epochs))
        print('-' * 10)
        
        loss_train = 0
        loss_val = 0
        acc_train = 0
        acc_val = 0
        
        model_tr.train(True)

        mm = [train_data_loader, valid_data_loader]

        for dt_loader in mm:
          print(dt_loader)

          for i, data in enumerate(dt_loader):
            if i % 100 == 0:
                print("\rTraining batch {}/{}".format(i, train_batches), end='', flush=True)
                
            # took one folder.imagas
            inputs, labels = data
            
            # zero out the gradients
            optimizer.zero_grad()
            
            #
            outputs = model_tr(inputs)
            
            # Stores in 'preds' a vector of positions of maximum values in the tensor
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            loss.backward() # Back Propagation
            optimizer.step() # Gardient Descent
            
            # Calculate loss & accuracy
            loss_train += loss.data
            acc_train += torch.sum(preds == labels.data)
            
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()
        
          print()
          
          if dt_loader is train_data_loader:
            avg_loss = loss_train / len(train_data_loader)
            avg_acc = acc_train / len(train_data_loader)
            
            model_tr.train(False)
            model_tr.eval()

          else:
            avg_loss_val = loss_val / len(valid_data_loader)
            avg_acc_val = acc_val / len(valid_data_loader)

        
        print()
        print("Epoch {} result: ".format(epoch))
        print("Avg loss (train): {:.4f}".format(avg_loss))
        print("Avg acc (train): {:.4f}".format(avg_acc))
        print("Avg loss (val): {:.4f}".format(avg_loss_val))
        print("Avg acc (val): {:.4f}".format(avg_acc_val))
        print('-' * 10)
        print()
        
        if avg_acc_val > best_acc:
            best_acc = avg_acc_val
            best_model_wts = copy.deepcopy(model_tr.state_dict())
        
    elapsed_time = time.time() - since
    print()
    print("Training completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Best acc: {:.4f}".format(best_acc))
    
    model_tr.load_state_dict(best_model_wts)
    return model_tr

# Buld Model

In [ ]:
bird_model = train_model(model_ft, criterion, optimazer, None , 1)

# New train

In [ ]:
train_batches = len(train_data_loader)
val_batches = len(valid_data_loader)

for epoch in range(epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    print("Epoch {}/{}".format(epoch, epochs))
    print('-' * 10)

    for i, data in enumerate(train_data_loader):

        if i % 100 == 0:
          print("\rTraining batch {}/{}".format(i, train_batches), end='', flush=True)

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimazer.zero_grad()



        # forward + backward + optimize
        outputs = model_ft(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimazer.step()

        # print statistics
        # if i % 2000 == 1999:    # print every 2000 mini-batches
        running_loss += loss.item()
        print(f'epoch: {epoch + 1} loss: {loss}')
        running_loss = 0.0
        if i == 100: 
          break
    break

print('Finished Training')